In [ ]:
!pip install ultralytics opencv-python pybullet matplotlib

In [ ]:
import cv2
import numpy as np
import pybullet as p
import pybullet_data
from ultralytics import YOLO
import matplotlib.pyplot as plt

# Load YOLOv8 model
model = YOLO("yolov8n.pt")  # Replace with your trained model for billiard balls

# Initialize PyBullet
p.connect(p.GUI)  # Use DIRECT for headless mode
p.setAdditionalSearchPath(pybullet_data.getDataPath())
plane_id = p.loadURDF("plane.urdf")
table_id = p.createCollisionShape(p.GEOM_BOX, halfExtents=[1.5, 0.75, 0.01])  # Approx table size
p.createMultiBody(baseCollisionShapeIndex=table_id, basePosition=[0, 0, -0.01])

# Ball properties
BALL_RADIUS = 0.028
BALL_MASS = 0.17

# Add a ball to the simulation
def add_ball(position, velocity):
    ball_id = p.createCollisionShape(p.GEOM_SPHERE, radius=BALL_RADIUS)
    body_id = p.createMultiBody(
        baseMass=BALL_MASS,
        baseCollisionShapeIndex=ball_id,
        basePosition=position,
    )
    p.resetBaseVelocity(body_id, linearVelocity=velocity)
    return body_id

# Process video
cap = cv2.VideoCapture("/content/drive/MyDrive/billiard_game.mp4")  # Replace with your video file
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO object detection
    results = model(frame, conf=0.5)
    balls = []
    for result in results.xyxy[0]:  # Each detection
        x1, y1, x2, y2, conf, cls = result
        cx, cy = (x1 + x2) / 2, (y1 + y2) / 2
        balls.append((cx, cy))

    # Map pixel positions to PyBullet coordinates
    for ball in balls:
        px, py = ball
        position = [(px / frame_width) * 2.84 - 1.42, (py / frame_height) * 1.42 - 0.71, BALL_RADIUS]
        add_ball(position, [0, 0, 0])  # Add with zero velocity for now

    # Simulate trajectory
    for _ in range(240):  # Simulate for 1 second at 240 Hz
        p.stepSimulation()

    # Visualize trajectory
    ball_positions = [
        p.getBasePositionAndOrientation(body_id)[0][:2]
        for body_id in range(p.getNumBodies())
    ]
    for path in ball_positions:
        plt.scatter(*path, label="Ball Trajectory")
    plt.legend()
    plt.show()
    break  # Process only the first frame

cap.release()
cv2.destroyAllWindows()
p.disconnect()
